In [ ]:
import requests
from pystac_client import Client
from pathlib import Path

URL_LANDING_PAGE = "https://catalog.maap.eo.esa.int/catalogue/"
api = Client.open(URL_LANDING_PAGE) # Connection to the catalog

# Search product by ID using the STAC API

In [ ]:
ids = ["BIO_S2_SCS__1S_20251216T034800_20251216T034815_T_G01_M01_C02_T017_F289_01_DJQGAN"]

results = api.search(
    method = 'GET',   
    collections = 'BiomassLevel1a',
    ids = ids
)

In [ ]:
items = results.item_collection()
items[0]

In [ ]:
url_abs = items[0].assets["enclosure_i_abs_tiff"].href
url_phase = items[0].assets["enclosure_i_phase_tiff"].href

# Request a token for product access

In [ ]:
def load_credentials(file_path):
    """Read key-value pairs from a credentials file into a dictionary."""
    creds = {}
    if not file_path.exists():
        raise FileNotFoundError(f"Credentials file not found: {file_path}")
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            if "=" not in line:
                continue
            key, value = line.split("=", 1)
            creds[key.strip()] = value.strip()
    return creds


def get_token(credential_file_path):
    """Use OFFLINE_TOKEN to fetch a short-lived access token."""
    creds = load_credentials(Path(credential_file_path))

    OFFLINE_TOKEN = creds.get("OFFLINE_TOKEN")
    CLIENT_ID = creds.get("CLIENT_ID")
    CLIENT_SECRET = creds.get("CLIENT_SECRET")

    if not all([OFFLINE_TOKEN, CLIENT_ID, CLIENT_SECRET]):
        raise ValueError("Missing OFFLINE_TOKEN, CLIENT_ID, or CLIENT_SECRET in credentials file")

    url = "https://iam.maap.eo.esa.int/realms/esa-maap/protocol/openid-connect/token"
    data = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "refresh_token",
        "refresh_token": OFFLINE_TOKEN,
        "scope": "offline_access openid"
    }

    response = requests.post(url, data=data)
    response.raise_for_status()

    response_json = response.json()
    access_token = response_json.get('access_token')

    if not access_token:
        raise RuntimeError("Failed to retrieve access token from IAM response")

    return access_token

# Access product raster and process on the fly

In [ ]:
# Authentication
# token = get_token("./credentials.txt")
token = get_token("/data/credentials.txt")

In [ ]:
import rioxarray as riox
import rasterio as rio

# Datasets lazy loading
with rio.Env(
    GDAL_HTTP_HEADERS=f"Authorization: Bearer {token}",
    GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
    GDAL_NUM_THREADS="ALL_CPUS",
):
    da_abs = riox.open_rasterio(url_abs, chunks={})
    da_phase = riox.open_rasterio(url_phase, chunks={})

In [ ]:
import numpy as np
import xarray as xr

# Reconstruct SLC images
slc = da_abs * np.exp(1j * da_phase)
dims = ("y", "x")
# Extract polSAR channels and create a PolSARpro S matrix
S = xr.Dataset(
    {
        "hh": (dims, slc[0].data),
        "hv": (dims, slc[1].data),
        "vh": (dims, slc[2].data),
        "vv": (dims, slc[3].data),
    },
    coords={"y": np.arange(slc.shape[1]), "x": np.arange(slc.shape[2])},
    attrs={"poltype": "S", "description": "Scattering matrix"},
).chunk("auto")

In [ ]:
from polsarpro.decompositions import freeman
from dask.diagnostics import ProgressBar
import os

out_file = "/data/psp/res/freeman_biomass.nc"
if os.path.exists(out_file):
    os.remove(out_file)

# HTTP warnings may be ignored
with ProgressBar():
    freeman(S, [7, 7]).to_netcdf(out_file)

In [ ]:
out_file = "/data/psp/res/freeman_biomass.nc"
out = xr.open_dataset(out_file)

In [ ]:
out.volume.plot.imshow(vmax=1)